In [2]:
import requests, re, time, pymysql, jieba
import jieba.posseg as postag
tables = [ 'wms', 'swb','nyj', 'zscqj', 'jbw', 'zjh', 'yjh', 'rmyh']
lim = 10

In [4]:
# db = pymysql.connect(host='119.29.190.115', user='guest', password='njuacmicpc',
#                          port=3306, db='gov', write_timeout = 6, read_timeout = 6)
# cursor = db.cursor()
# def create_table(cursor, name):
#     sql = 'create table if not exists {} (pos VARCHAR(16), title VARCHAR(512), link VARCHAR(512), date VARCHAR(32), origin VARCHAR(8), type VARCHAR(8),  PRIMARY KEY (link))'.format(name)
#     cursor.execute(sql)

# create_table(cursor, 'summary')

In [26]:
def get_sql(table = 'jbw', dec = 0):
    month = time.strftime('%Y-%m', time.localtime(time.time() - 24 * 3600 * 30 * dec))
    sql = "SELECT title FROM gov.{} WHERE date >= '{}-01' and date <= '{}-31'".format(table, month, month)
    return sql

def get_sentense_dict():
    db = pymysql.connect(host='119.29.190.115', user='guest', password='njuacmicpc',
                         port=3306, db='gov', write_timeout = 6, read_timeout = 6)
    cursor = db.cursor()
    dic_by_num = {}
    for dec in range(lim):
        dic_by_num[dec] = []
        for table in tables:
            sql = get_sql(table, dec)
            try:
                cursor.execute(sql)
                db.commit()
                for row in cursor.fetchall():
                    if len(row) > 0:
                        dic_by_num[dec].append(row[0])
            except Exception as err:
                print("Error {}".format(err))
                time.sleep(5)
    db.close()
    return dic_by_num

def aggregate_tables():
    db = pymysql.connect(host='119.29.190.115', user='guest', password='njuacmicpc',
                         port=3306, db='gov', write_timeout = 6, read_timeout = 6)
    cursor = db.cursor()
    
    input_list = []
    for table in tables:
        sql = 'SELECT * FROM gov.{}'.format(table)
        try:
            cursor.execute(sql)
            db.commit()
            for row in cursor.fetchall():
                if len(row) > 0:
                    while len(row) < 4:
                        row.append('')
                    input_list.append({'pos': row[0], 'title': row[1], 'link': row[2], 'date': row[3], 'origin': table})
        except Exception as err:
            print("Error {}".format(err))
            time.sleep(5)
    db.close()
    
    return input_list

def insert_summary_table(input_list):
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc',
                         port=3306, db='gov', write_timeout = 6, read_timeout = 6)
    cursor = db.cursor()
    for dic in input_list:
        try:
            sql_insert(db=db,cursor=cursor,data=dic,table='summary')
        except Exception as err:
            print("Error {}".format(err))
            time.sleep(5)
    db.close()

def get_words(sentence_list):
    words = []
    jieba.enable_parallel(8)
    for raw in sentence_list:
        result = postag.cut(raw)
        raw = [x.word for x in result if (len(x.word) > 1 and 'n' in x.flag)]
        words += raw           
    return words

def get_word_count(lst, stop_words = None, re_stop_letter = None):
    count = {}
    for x in lst:
        if ((stop_words == None or x not in stop_words) and 
           (re_stop_letter == None or re.search(re_stop_letter, x) == None)):
            if x in count:
                count[x] += 1
            else:
                count[x] = 1
    return count

def sql_insert(db, cursor, table, data):
    keys = ', '.join(data.keys())
    values = ', '.join(['%s'] * len(data))
    sql = 'insert into {table}({keys}) values ({values}) on duplicate key update'.format(table=table, 
            keys=keys, values=values)
    update = ','.join([" {key} = %s".format(key=key) for key in data])
    sql += update
    try:
        if cursor.execute(sql, tuple(data.values())*2):
            print('Successful')
            db.commit()
        else:
            print ('Nothing to do')
    except:
        print ('Failed')
        db.rollback()

def insert_wordcnt(sentense_dict):
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc',
                         port=3306, db='gov', write_timeout = 6, read_timeout = 6)
    cursor = db.cursor()
    for dec in range(lim):
        month = time.strftime('%Y-%m', time.localtime(time.time() - 24 * 3600 * 30 * dec))
        word_counts = get_word_count(get_words(sentense_dict[dec]))
        word_dict_list = [ {'word':x[0], 'month': month, 'tf': int(x[1])} for x in word_counts.items()]
        try:
            for dic in word_dict_list:
                sql_insert(db=db,cursor=cursor,data=dic,table='word_tf')
        except Exception as err:
            print("Error {}".format(err))
            time.sleep(5)
    
    db.close()

def dic_with_name(dic, name):
    dic['type'] = name
    return dic

def insert_other_table(input_list, regex_dict):
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc',
                         port=3306, db='gov', write_timeout = 6, read_timeout = 6)
    cursor = db.cursor()
    table_list = ['TZGG','HYPX','ZCFB','ZCJD','ZFSJ','YWDD']
    
    input_list_with_type = []
    
    for dic in input_list:
        tit = dic['title']
        try:
            is_inserted = False
            for name in table_list[:-1]:
                if tit != None and re.search(regex_dict[name], tit) != None:
                    sql_insert(db=db,cursor=cursor,data=dic,table=name)
                    input_list_with_type.append(dic_with_name(dic, name))
#                     print (dic_with_name(dic,name))
                    is_inserted = True
                    
            if is_inserted == False or re.search(regex_dict[table_list[-1]], tit) != None:
                sql_insert(db=db,cursor=cursor,data=dic,table=table_list[-1])
                input_list_with_type.append(dic_with_name(dic,table_list[-1]))
#                 print (dic_with_name(dic,table_list[-1]))
                
        except Exception as err:
            print("Error {}".format(err))
            time.sleep(5)
            
    db.close()
    
    return input_list_with_type

In [27]:
#通知公告(TZGG)：'通知|公告|号令|函|通报|公示|问卷'
#会议培训(HYPX)：'培训|开班|学习|会议|会$|召开|的办法|的决定'
#政策发布(ZCFB)：'发布|印发|决定|意见
#政策解读(ZCJD)：'解读|谈|答'
#政府数据(ZFSJ)：'年.*表$|报$|报告|月报|季报|年报$
#要闻导读(YWDD)：'活动|开展' (以及前面未出现的所有)
regex = dict()
regex['TZGG'] = re.compile('通知|公告|号令|函|通报|公示|问卷')
regex['HYPX'] = re.compile('培训|开班|学习|会议|会$|召开|的办法|的决定')
regex['ZCFB'] = re.compile('发布|印发|决定|意见')
regex['ZCJD'] = re.compile('解读|谈|答')
regex['ZFSJ'] = re.compile('年.*表$|报$|报告|月报|季报|年报')
regex['YWDD'] = re.compile('活动|开展')

In [28]:
input_list = aggregate_tables()

In [29]:
input_list_with_type = insert_other_table(input_list, regex)

Nothing to do
{'pos': 'guangdong', 'title': '中国与圣多美和普林西比关于互免持外交、公务护照（特别公务护照）人员签证的协定即将生效', 'link': 'http://go.gdcom.gov.cn/article.php?typeid=34&contentId=10762', 'date': '2018-02-01', 'origin': 'wms', 'type': 'YWDD'}
Nothing to do
{'pos': 'guangdong', 'title': '中外互免签证协议一览表（2018年06月18日更新）', 'link': 'http://go.gdcom.gov.cn/article.php?typeid=34&contentId=13482', 'date': '2018-10-30', 'origin': 'wms', 'type': 'YWDD'}
Nothing to do
{'pos': 'guangdong', 'title': '中华人民共和国政府和巴林王国政府关于互免持外交、公务和特别护照人员签证的协定已生效', 'link': 'http://go.gdcom.gov.cn/article.php?typeid=34&contentId=13483', 'date': '2018-10-30', 'origin': 'wms', 'type': 'YWDD'}
Nothing to do
{'pos': 'guangdong', 'title': '保监会设禁令 监管严防险企海外融资变相骗汇', 'link': 'http://go.gdcom.gov.cn/article.php?typeid=36&contentId=12314', 'date': '2018-05-21', 'origin': 'wms', 'type': 'YWDD'}
Nothing to do
{'pos': 'guangdong', 'title': '外汇局严厉查处外汇违规行为 维护外汇市场健康良性秩序', 'link': 'http://go.gdcom.gov.cn/article.php?typeid=36&contentId=12967', 'date': '2018-07-27', 'o

Nothing to do
{'pos': 'jiangsu', 'title': '商务部公告2015年第79号 公布《2016年出口许可证管理货物分级发证目录》', 'link': 'http://swt.jiangsu.gov.cn/art/2016/1/26/art_57692_7100546.html', 'date': '2016-01-26', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '商务部关于印发《2016年度甘草及甘草制品出口配额第一次招标中标企业名单及其额度表》的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2016/1/7/art_57692_7100540.html', 'date': '2016-01-07', 'origin': 'wms', 'type': 'TZGG'}
Failed
{'pos': 'jiangsu', 'title': '商务部关于印发《2016年度甘草及甘草制品出口配额第一次招标中标企业名单及其额度表》的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2016/1/7/art_57692_7100540.html', 'date': '2016-01-07', 'origin': 'wms', 'type': 'ZCFB'}
Nothing to do
{'pos': 'jiangsu', 'title': '商务部公告2015年第73号 公布2016年度符合铁合金出口许可条件的企业名单', 'link': 'http://swt.jiangsu.gov.cn/art/2016/1/7/art_57692_7100541.html', 'date': '2016-01-07', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '商务部关于公布2016年第一批工业品出口配额的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2016/1/7/art_57692_7100542.html'

Nothing to do
{'pos': 'jiangsu', 'title': '关于组织参加亚洲国际机床展览会的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2018/1/15/art_57691_7389894.html', 'date': '2018-01-15', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '关于组织参加美国国际医疗展的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2018/1/15/art_57691_7392835.html', 'date': '2018-01-15', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '关于组织参加中国消费品（俄罗斯）品牌展的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2018/1/3/art_57691_7226863.html', 'date': '2018-01-03', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '关于组织参加第二十二届江苏省出口商品展览会（日本·大阪）的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2018/1/9/art_57691_7280024.html', 'date': '2018-01-09', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'jiangsu', 'title': '关于组织参加2018年中国品牌商品波兰展的通知', 'link': 'http://swt.jiangsu.gov.cn/art/2018/1/9/art_57691_7280391.html', 'date': '2018-01-09', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'po

Nothing to do
{'pos': 'china', 'title': '商务部关于补充公布部分工业品出口企业名单及2018年出口配额的通知', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/ax/201809/20180902782678.shtml', 'date': '2018-09-04', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部公告2018年78号 2019年食糖进口关税配额申请和分配细则', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/ax/201810/20181002794869.shtml', 'date': '2018-09-30', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部\u3000海关总署公告2018年第82号（公布货物进口许可证件申领和通关无纸化作业有关事项）', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/ax/201810/20181002794907.shtml', 'date': '2018-10-12', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部公告2018年第87号 公布2019年货物出口配额管理有关事项', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/ax/201810/20181002801178.shtml', 'date': '2018-10-30', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部办公厅关于《对外贸易经营者备案登记表》使用法人和其他组织统一社会信用代码的通知', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/

Nothing to do
{'pos': 'china', 'title': '商务部办公厅关于加工贸易项下白银出口审批业务实行网上办理的通知', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/g/201810/20181002797486.shtml', 'date': '2018-10-17', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部公告2018年79号 2019年羊毛、毛条进口关税配额管理实施细则', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/g/201810/20181002797940.shtml', 'date': '2018-09-30', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '商务部公告2018年第87号 公布2019年货物出口配额管理有关事项', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/g/201810/20181002801178.shtml', 'date': '2018-10-30', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'china', 'title': '关于2018年第二批燃料油非国营贸易进口申请企业有关情况的公示', 'link': 'http://wms.mofcom.gov.cn/article/zcfb/g/201810/20181002801392.shtml', 'date': '2018-10-31', 'origin': 'wms', 'type': 'TZGG'}
Nothing to do
{'pos': 'zhejiang', 'title': '100家浙江省加工贸易创新示范企业政策培训班暨授牌仪式圆满成...', 'link': 'http://www.zcom.gov.cn/art/2018/1/18/art_1416072_15259343.html', 'd

...

Nothing to do
{'pos': 'guangdong', 'title': '2017年12月广东省金融机构人民币存贷款主要项目表', 'link': 'http://guangzhou.pbc.gov.cn/guangzhou/129138/3475255/index.html', 'date': '2018-01-30', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'guangdong', 'title': '2017年12月广东省金融机构外汇存贷款主要项目表', 'link': 'http://guangzhou.pbc.gov.cn/guangzhou/129138/3475268/index.html', 'date': '2018-01-30', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'guangdong', 'title': '2018年1月广东省金融机构本外币存贷款主要项目表', 'link': 'http://guangzhou.pbc.gov.cn/guangzhou/129138/3495807/index.html', 'date': '2018-02-27', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'guangdong', 'title': '2018年1月广东省金融机构人民币存贷款主要项目表', 'link': 'http://guangzhou.pbc.gov.cn/guangzhou/129138/3495812/index.html', 'date': '2018-02-27', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'guangdong', 'title': '2018年1月广东省金融机构外币存贷款主要项目表', 'link': 'http://guangzhou.pbc.gov.cn/guangzhou/129138/3495817/index.html', 'date': '2018-02-27', 'origin': 'rmy

Nothing to do
{'pos': 'zhejiang', 'title': '2018年5月浙江省金融机构本外币存贷款数据表', 'link': 'http://hangzhou.pbc.gov.cn/hangzhou/125249/3567760/index.html', 'date': '2018-06-28', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'zhejiang', 'title': '2018年6月浙江省金融机构本外币存贷款数据表', 'link': 'http://hangzhou.pbc.gov.cn/hangzhou/125249/3586645/index.html', 'date': '2018-07-25', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'zhejiang', 'title': '2018年7月浙江省金融机构本外币存贷款数据表', 'link': 'http://hangzhou.pbc.gov.cn/hangzhou/125249/3610623/index.html', 'date': '2018-08-24', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'zhejiang', 'title': '2018年8月浙江省金融机构本外币存贷款数据表', 'link': 'http://hangzhou.pbc.gov.cn/hangzhou/125249/3638206/index.html', 'date': '2018-09-30', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'zhejiang', 'title': '新年首个工作日赴南湖瞻仰学习 人行杭州中支举行“不忘初心 牢记使命”主题党日活动', 'link': 'http://hangzhou.pbc.gov.cn/hangzhou/125264/3455015/index.html', 'date': '2018-01-03', 'origin': 'rmyh', 'typ

Nothing to do
{'pos': 'jiangsu', 'title': '人民银行盐城市中心支行精准发力服务实体经济', 'link': 'http://nanjing.pbc.gov.cn/nanjing/117525/3625334/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'jiangsu', 'title': '人民银行徐州市中心支行实现全账户种类业务核准“一站式”办理', 'link': 'http://nanjing.pbc.gov.cn/nanjing/117525/3625337/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'jiangsu', 'title': '宿迁市成立省内首家地市级金融消费权益保护协会', 'link': 'http://nanjing.pbc.gov.cn/nanjing/117525/3639291/index.html', 'date': '', 'origin': 'rmyh', 'type': 'HYPX'}
Nothing to do
{'pos': 'jiangsu', 'title': '人民银行扬州市中心支行为江苏省园博会提供全方位反假货币服务', 'link': 'http://nanjing.pbc.gov.cn/nanjing/117525/3640853/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'jiangsu', 'title': '人民银行徐州市中心支行强化警税银反洗钱合作机制', 'link': 'http://nanjing.pbc.gov.cn/nanjing/117525/3645351/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'jiangsu', 'title': '人民银行无锡市中心支行推动成立无锡市

Nothing to do
{'pos': 'china', 'title': '2018年7月金融统计数据报告', 'link': 'http://www.pbc.gov.cn/diaochatongjisi/116219/116225/3603624/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'china', 'title': '2018年8月末社会融资规模存量统计数据报告', 'link': 'http://www.pbc.gov.cn/diaochatongjisi/116219/116225/3625643/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'china', 'title': '2018年8月社会融资规模增量统计数据报告', 'link': 'http://www.pbc.gov.cn/diaochatongjisi/116219/116225/3625696/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'china', 'title': '2018年8月金融统计数据报告', 'link': 'http://www.pbc.gov.cn/diaochatongjisi/116219/116225/3625701/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'china', 'title': '2018年前三季度社会融资规模增量统计数据报告', 'link': 'http://www.pbc.gov.cn/diaochatongjisi/116219/116225/3646124/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZFSJ'}
Nothing to do
{'pos': 'china', 'title': '201

Nothing to do
{'pos': 'china', 'title': '中国人民银行 中国银行业监督管理委员会令〔2017〕第2...', 'link': 'http://www.pbc.gov.cn/tiaofasi/144941/144957/3590858/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'china', 'title': '中国人民银行令〔2017〕第3号（应收账款质押登记办法）', 'link': 'http://www.pbc.gov.cn/tiaofasi/144941/144957/3590864/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'china', 'title': '中国人民银行令〔2018〕第2号（中国人民银行关于修改《...', 'link': 'http://www.pbc.gov.cn/tiaofasi/144941/144957/3596827/index.html', 'date': '', 'origin': 'rmyh', 'type': 'YWDD'}
Nothing to do
{'pos': 'china', 'title': '中国人民银行关于印发《支付结算办法》的通知（银发〔...', 'link': 'http://www.pbc.gov.cn/tiaofasi/144941/144957/3601571/index.html', 'date': '', 'origin': 'rmyh', 'type': 'TZGG'}
Failed
{'pos': 'china', 'title': '中国人民银行关于印发《支付结算办法》的通知（银发〔...', 'link': 'http://www.pbc.gov.cn/tiaofasi/144941/144957/3601571/index.html', 'date': '', 'origin': 'rmyh', 'type': 'ZCFB'}
Nothing to do
{'pos': 'china', 'titl

In [30]:
insert_summary_table(input_list_with_type)

Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Nothing to do
Successful
Nothing to do
Nothing to do
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to d

Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Nothing to do
Nothing to do
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Nothing to do
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing t

Successful
Successful
Nothing to do
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Nothing to do
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Successful
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Successful
Successful
Nothing to do
Successful
Successful
Successful
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Nothing to do
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Nothing to do
Succ

In [14]:
sdict = get_sentense_dict()

In [15]:
insert_wordcnt(sdict)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.899 seconds.
Prefix dict has been built succesfully.


Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Nothing to do
Nothing to do
Nothing to do
Successful
Nothing to do
Nothing to do
Nothing to do
Successful
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Successful
Successful
Nothing to do
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothin

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothin

Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothin

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing 

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothin

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothin

Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
Nothing to do
